## Start

In [19]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



In [ ]:
import boto3



___
### Tags: #Tables

### Links: 

[[requests_st_support]]

[[intercoms_dir_asgard]]

[[intercoms_st_partner]]

[[companies_dir_partner]]
___

In [20]:
query_text = """--sql
CREATE TABLE db1.t_requests_st_support
    (
        `report_date` Date,
        `ticket_id` Int32,
        `intercom_id` String,
        `intercom_uuid` String,
        `camera_id` String,
        `server_stream_url` String,
        `detailed_category` String,
        `category` String,
        `subcategory` String,
        `maincategory` String,  
        `version_os` String,
        `version_app` String,
        `partner_uuid` String,
        `company_name` String,
        `partner_lk` String,
        `com_d_par.company_name` String,
        `device_type` String
    )
    ENGINE = MergeTree()
    ORDER BY report_date
"""

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_requests_st_support_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_requests_st_support AS
    WITH t1 AS(
SELECT 
    req_s_sup.`report_date` AS report_date, 
    `ticket_id`, 
    `intercom_id`, 
    int_d_as.`intercom_uuid` AS intercom_uuid,
    `camera_id`,
    `server_stream_url`, 
    replaceAll(trim(arrayJoin(splitByChar(',',i`category`))),'.','') AS `category`, 
    `version_os`, 
    `version_app`,
    int_s_par.`partner_uuid` AS partner_uuid,
    `company_name`,
    `partner_lk`,
    `device_type`,
FROM `db1`.`requests_st_support_ch` AS req_s_sup
LEFT JOIN db1.intercoms_dir_asgard_ch AS int_d_as ON int_d_as.motherboard_id = req_s_sup.intercom_id
LEFT JOIN db1.intercoms_st_partner_ch AS int_s_par 
    ON  int_s_par.intercom_uuid = int_d_as.`intercom_uuid`
    AND int_s_par.report_date = req_s_sup.`report_date`
LEFT JOIN db1.companies_dir_partner_ch AS com_d_par
    ON int_s_par.partner_uuid = com_d_par.partner_uuid
   	)
--
SELECT
	report_date,
	ticket_id,
	intercom_id,
	intercom_uuid,
	camera_id,
    server_stream_url,
    category,
    splitByChar(':',category)[2] AS `detailed_category`,
    if(splitByChar('|',splitByChar(':',category)[1])[2] = '','Main',splitByChar('|',splitByChar(':',category)[1])[2])  AS `subcategory`, 
    splitByChar('|',splitByChar(':',category)[1])[1] AS `maincategory`, 
    version_os, 
    version_app,
    partner_uuid,
    company_name,
    partner_lk,
    device_type,
FROM t1
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

___
## Tools
___
### query


In [22]:
query_text = """--sql
    SELECT
        *
    FROM db1.t_requests_st_support
    ORDER BY report_date desc
    limit 100
    """

ch.query_run(query_text)


report_date,ticket_id,intercom_id,intercom_uuid,camera_id,server_stream_url,detailed_category,category,subcategory,maincategory,version_os,version_app,partner_uuid,company_name,partner_lk,com_d_par.company_name,device_type
date,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2025-07-22,197109,"""2018111747""","""c6179773-031f-48ec-bd00-2d7687…","""""","""v08.sputnik.systems""",""" глубина архива""","""MOB|Info: глубина архива""","""Info""","""MOB""","""iOS""","""4.32.0(7)""","""0e7236ac-b8bd-4ab0-8634-d165ad…","""Цифрал Сервис УК""","""120345""","""""","""intercom"""
2025-07-22,197102,"""2101115815""","""85066ae3-7bff-489f-bb22-c12eb6…","""""","""msk-v04.sputnik.systems""",""" оффлайн""","""NET: оффлайн""","""Main""","""NET""","""iOS""","""4.34.0(12)""","""71c6b362-bf23-43c5-8762-ac0f12…","""Частный дом""","""128024""","""""","""intercom"""
2025-07-22,197102,"""2101115815""","""85066ae3-7bff-489f-bb22-c12eb6…","""""","""msk-v04.sputnik.systems""",""" проблема с оплатой""","""MOB|Info: проблема с оплатой""","""Info""","""MOB""","""iOS""","""4.34.0(12)""","""71c6b362-bf23-43c5-8762-ac0f12…","""Частный дом""","""128024""","""""","""intercom"""
2025-07-22,197100,"""1944102468""","""89b41829-d02e-4f82-aefd-5dff8e…","""""","""msk-v01.sputnik.systems""",""" сменить в приложении номер те…","""MOB|Info: сменить в приложении…","""Info""","""MOB""","""Android""","""6.21.1(417)""","""720c4488-0cd0-40f3-bc88-192f47…","""DM SEVICE CONTROL SRL""","""120750""","""""","""intercom"""
2025-07-22,197104,"""2428254675""","""3ea617d3-70e6-43da-93d0-7a1f02…","""""","""msk-v10.sputnik.systems""","""""","""Другое""","""Main""","""Другое""","""iOS""","""4.34.0(12)""","""684986ca-87e7-4d53-8a3c-8cb47a…","""ДОМИНАНТ КРД""","""125759""","""""","""intercom"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-07-22,197008,"""1944104592""","""800dbca3-8eab-425d-b144-da0774…","""""","""msk-v04.sputnik.systems""",""" оффлайн""","""NET: оффлайн""","""Main""","""NET""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""
2025-07-22,197008,"""1944104592""","""800dbca3-8eab-425d-b144-da0774…","""""","""msk-v04.sputnik.systems""",""" видео не работает""","""MOB|Video: видео не работает""","""Video""","""MOB""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""
2025-07-22,197029,"""2018112269""","""cdc93a98-f790-40b0-abde-d87c7c…","""""","""v07.sputnik.systems""",""" видеоархива нет/недоступен/не…","""MOB|Storage: видеоархива нет/н…","""Storage""","""MOB""","""Android""","""6.21.1(417)""","""e7e86c75-fc8b-4710-839d-56ad86…","""Цифрал Тольятти""","""152762""","""""","""intercom"""


### delete a part


In [ ]:
query_text = """--sql
    ALTER TABLE db1.t_requests_st_support DELETE WHERE report_date = '2025-07-17'
    """

ch.query_run(query_text)


### drop mv

In [17]:
query_text = """--sql
    DROP TABLE db1.t_requests_st_support_mv
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘


### drop ch

In [18]:
query_text = """--sql
    DROP TABLE db1.t_requests_st_support
    """

ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

### refresh mv

In [5]:
query_text = """
SYSTEM REFRESH VIEW db1.t_requests_st_support_mv
"""

ch.query_run(query_text)

elapsed_ns,query_id
i64,str
762558,"""b8a0939c-8390-471b-99f8-16ccf5…"
